In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/ChatBot

[Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/ChatBot'
/content


In [ ]:
ls

'Dataset Kampus Merdeka.json'   Kadek_chatbot.ipynb


In [ ]:
# prepare dataset
import json
import pandas as pd

with open("Dataset Kampus Merdeka.json") as data_file:
    data = json.load(data_file)

FileNotFoundError: ignored

In [ ]:
def preprocess_string(string):
    string = string.lower()
    exclude = set(punctuation)
    string = ''.join(ch for ch in string if ch not in exclude)
    return string
  

In [ ]:
def chat(model):
    print("Anda akan dihubungkan ke Kadekbot, mohon ditunggu")
    exit = False
    while not exit:
        inp = input("Anda : ")
        inp = preprocess_string(inp)
        prob = model.predict([inp])
        results = le.classes_[prob.argmax()]
        if prob.max() < 0.2:
            print("Bot : Maaf kak, kami tidak mengerti")
        else:
            for tg in data['intents']:
                if tg['tag'] == results:
                    responses = tg['responses']
            if results == 'bye':
                exit = True
                print("END CHAT")
            print(f"Bot : {random.choice(responses)}")

In [ ]:
text_input = []
intents = []
responses = []
ignore_letters = ['!', '?', ',', '.', '[]']

for intent in data['intents']:
    for input in intent['input']:
        text_input.append(input)
        intents.append(intent['tag'])
        responses.append(intent['responses'])

df = pd.DataFrame({'text_input': text_input,
                    'intents': intents,
                   "responses":responses})

df.head()

,text_input,intents,responses
0,hallo,greeting,[Hai! Kadekbot di sini. Mau tau informasi tent...
1,hai,greeting,[Hai! Kadekbot di sini. Mau tau informasi tent...
2,halo,greeting,[Hai! Kadekbot di sini. Mau tau informasi tent...
3,hei,greeting,[Hai! Kadekbot di sini. Mau tau informasi tent...
4,hi,greeting,[Hai! Kadekbot di sini. Mau tau informasi tent...


In [ ]:
df.intents.value_counts()

greeting                                        15
program_kampus_merdeka                          11
goodbye                                          9
terimakasih                                      5
persyaratan_IISMA                                2
manfaat_PMMB                                     1
periode_dan_pendaftaran_pertukaran_mahasiswa     1
periode_dan_pendaftaran_IISMA                    1
manfaat_IISMA                                    1
mitra_IISMA                                      1
penjelasan_pertukaran_mahasiswa                  1
persyaratan_pertukaran_mahasiswa                 1
manfaat_pertukaran_mahasiswa                     1
periode_dan_pendaftaran_PMMB                     1
penjelasan_IISMA                                 1
persyaratan_pejuang_muda                         1
periode_dan_pendaftaran_pejuang_muda             1
manfaat_pejuang_muda                             1
penjelasan_PMMB                                  1
penjelasan_pejuang_muda        

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# lemmaztize and lower each word and remove duplicates
intents = [lemmatizer.lemmatize(w.lower()) for w in intents if w not in ignore_letters]
intents = sorted(list(set(intents)))
text_input = sorted(list(set(text_input)))
print (len(responses), "responses")
print (len(text_input), "input", text_input)
print (len(intents), "unique lemmatized tag", intents)

75 responses
74 input ['Apa itu IISMA?', 'Apa itu Kampus Mengajar?', 'Apa itu Kampus Merdeka?', 'Apa itu Magang Bersertifikat?', 'Apa itu PMMB?', 'Apa itu Pejuang Muda?', 'Apa itu Pertukaran Mahasiswa Merdeka?', 'Apa itu studi independen?', 'Apa persyaratan mengikuti Kampus Merdeka?', 'Apa saja manfaat apabila mengikuti IISMA?', 'Apa saja manfaat apabila mengikuti Kampus Mengajar?', 'Apa saja manfaat apabila mengikuti Magang Bersertifikat?', 'Apa saja manfaat apabila mengikuti PMMB?', 'Apa saja manfaat apabila mengikuti Pejuang Muda?', 'Apa saja manfaat apabila mengikuti Pertukaran Mahasiswa Merdeka?', 'Apa saja manfaat apabila mengikuti Studi Independen?', 'Apa saja mitra yang tergabung dalam Magang Bersertifikat?', 'Apa saja mitra yang tergabung dalam Studi Independen?', 'Apa saja persyaratan IISMA?', 'Apa saja persyaratan Kampus Mengajar?', 'Apa saja persyaratan Magang Bersertifikat?', 'Apa saja persyaratan Pejuang Muda?', 'Apa saja persyaratan Pertukaran Mahasiswa Merdeka?', 'Apa s

In [ ]:
#tokenize the data
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(df['text_input'])
train = tokenizer.texts_to_sequences(df['text_input'])
vocab_size = len(tokenizer.word_index)+1
print('vocab_size : {}'.format(vocab_size))

#apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train = pad_sequences(train)

vocab_size : 70


In [ ]:
# data cleansing
import string

# convert lowercase
df.text_input = df.text_input.apply(lambda x: x.lower())

# remove punctuation
exclude = set(string.punctuation)
df.text_input = df.text_input.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [ ]:
# label encoding
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

le = LabelEncoder()
y_train = le.fit_transform(df.intents)
y_train = to_categorical(y_train)

In [ ]:
all_vocab = []
length = []

for idx, row in df.iterrows():
    sent = row['text_input']
    [all_vocab.append(i) for i in sent.split()]
    length.append(len(sent.split()))

In [ ]:
len(all_vocab)

282

In [ ]:
max(length)

10

In [ ]:
len(set(all_vocab))

69

In [ ]:
from tensorflow.keras.layers import TextVectorization

max_vocab_length = 69
max_length = 10

text_vectorization = TextVectorization(max_tokens=max_vocab_length,
                                       standardize='lower_and_strip_punctuation',
                                       split='whitespace',
                                       ngrams=None,
                                       output_mode='int',
                                       output_sequence_length=max_length
                                       )


In [ ]:
text_vectorization.adapt(df.text_input)

In [ ]:
text_vectorization.get_vocabulary()

['',
 '[UNK]',
 'apa',
 'saja',
 'program',
 'merdeka',
 'kampus',
 'persyaratan',
 'mengikuti',
 'itu',
 'periode',
 'pelaksanaan',
 'mendaftar',
 'manfaat',
 'iisma',
 'dan',
 'cara',
 'bagaimana',
 'apabila',
 'studi',
 'magang',
 'independen',
 'pmmb',
 'pertukaran',
 'pejuang',
 'muda',
 'mengajar',
 'mahasiswa',
 'bersertifikat',
 'yang',
 'tergabung',
 'dalam',
 'you',
 'universitas',
 'terima',
 'mitra',
 'kasih',
 'bye',
 'tujuan',
 'tinggal',
 'thanks',
 'thank',
 'sore',
 'sis',
 'siang',
 'selamat',
 'see',
 'sampai',
 'pagi',
 'morning',
 'malam',
 'makasih',
 'kemensos',
 'kemendikbud',
 'kawan',
 'jumpa',
 'hy',
 'hi',
 'hei',
 'halo',
 'hallo',
 'hai',
 'good',
 'dah',
 'dadah',
 'byee',
 'bumn',
 'bro',
 'banyak']

In [ ]:
text_vectorization('Apa itu Kampus Merdeka?')

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([2, 9, 6, 5, 0, 0, 0, 0, 0, 0])>

In [ ]:
text_vectorization.get_vocabulary()[0]

''

In [ ]:
from tensorflow.keras.layers import Embedding
embedding = Embedding(input_dim=max_vocab_length,
                      output_dim=100,
                      embeddings_initializer="uniform",
                      input_length=max_length)

In [ ]:
import numpy as np
res_embed = embedding(np.array([[2, 9, 6, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]))
res_embed

<tf.Tensor: shape=(1, 22, 100), dtype=float32, numpy=
array([[[ 0.0324458 , -0.0319492 , -0.00137997, ...,  0.0210539 ,
          0.02299166,  0.0282779 ],
        [ 0.03256155, -0.04860206, -0.00965154, ...,  0.04435186,
          0.03360486, -0.03681355],
        [ 0.04808363,  0.0195342 , -0.04590813, ...,  0.03130461,
          0.01360308,  0.00620224],
        ...,
        [-0.03008989, -0.01659293,  0.02123663, ...,  0.0315851 ,
         -0.04615898, -0.03760599],
        [-0.03008989, -0.01659293,  0.02123663, ...,  0.0315851 ,
         -0.04615898, -0.03760599],
        [-0.03008989, -0.01659293,  0.02123663, ...,  0.0315851 ,
         -0.04615898, -0.03760599]]], dtype=float32)>

In [ ]:
# modelling
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, LSTM
inputs = Input(shape=(1,), dtype='string')
x = text_vectorization(inputs)
x = embedding(x)
x = LSTM(12)(x)
outputs = Dense(38, activation='softmax')(x)
model_lstm = Model(inputs, outputs, name="LSTM_model")

In [ ]:
# compile model
model_lstm.compile(loss='categorical_crossentropy',
                   optimizer='adam',
                   metrics=["accuracy"])

In [ ]:
model_lstm.fit(df.text_input,
               y_train,
               epochs=200,
               verbose=0)

In [ ]:
model_lstm.evaluate(df.text_input, y_train)

3/3 [==============================] - 1s 5ms/step - loss: 0.8610 - accuracy: 0.8933


[0.8609939813613892, 0.8933333158493042]

In [ ]:
model_lstm.save("bot_model.tf")

INFO:tensorflow:Assets written to: bot_model.tf/assets


INFO:tensorflow:Assets written to: bot_model.tf/assets


In [ ]:
import pickle
le_filename = open("label_encoder.pickle", "wb")
pickle.dump(le, le_filename)
le_filename.close()

In [ ]:
!unzip "/content/bot.zip" -d "/content/"

Archive:  /content/bot.zip
   creating: /content/bot_model.tf/assets/
  inflating: /content/bot_model.tf/saved_model.pb  
  inflating: /content/bot_model.tf/keras_metadata.pb  
  inflating: /content/bot_model.tf/variables/variables.index  
  inflating: /content/bot_model.tf/variables/variables.data-00000-of-00001  


In [ ]:
!pip install pyngrok==4.1.1
!pip install flask-ngrok
!ngrok authtoken '28MpMhvJzX5u3eKxDsCxhv6rYGD_2RCmZfYnBPBmS3LV4eQzZ'
from datetime import datetime
import pytz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
import time
from flask import Flask, request, render_template, send_from_directory
from flask_ngrok import run_with_ngrok

with open("Dataset Kampus Merdeka.json") as data_file:
    data = json.load(data_file)
model = load_model('bot_model.tf')
le_filename = open("label_encoder.pickle", "rb")
le = pickle.load(le_filename)
le_filename.close()


def preprocess_string(string):
    string = string.lower()
    exclude = set(punctuation)
    string = ''.join(ch for ch in string if ch not in exclude)
    return string

def chat(model, input_data):
    exit = False
    while not exit:
        inp = input_data
        inp = preprocess_string(inp)
        bot = ""
        prob = model.predict([inp])
        results = le.classes_[prob.argmax()]
        if prob.max() < 0.2:
            bot = "Maaf kak, aku ga ngerti"
        else:
            for tg in data['intents']:
                if tg['tag'] == results:
                    responses = tg['responses']
            if results == 'bye':
                exit = True
                print("END CHAT")
            bot = f"{random.choice(responses)}"
        return bot

app = Flask(__name__, template_folder='/content')
# The absolute path of the directory containing PDF files for users to download
app.config["CLIENT_PDF"] = "/content"
resultChat = []
resultBot = []
resultTime = []
now = datetime.now(pytz.timezone('Asia/Jakarta'))
time = now.strftime("%I:%M:%S %p %Z %a, %d %b %Y")
resultTime.append(time)
run_with_ngrok(app)   
@app.route("/")
def home():
    resultChat.clear()
    resultBot.clear()
    resultTime.clear()
    now = datetime.now(pytz.timezone('Asia/Jakarta'))
    time = now.strftime("%I:%M:%S %p %Z %a, %d %b %Y")
    resultTime.append(time)
    return render_template('index.html', len=0, time=resultTime)
@app.route("/predict", methods=['post'])
def predict():
    dataChat = request.form['input']
    now = datetime.now(pytz.timezone('Asia/Jakarta'))
    time = now.strftime("%I:%M:%S %p %Z %a, %d %b %Y")
    resultTime.append(time)
    print(dataChat)
    resultChat.append(dataChat)
    resultBot.append(chat(model, dataChat))
    return render_template('index.html', len = len(resultChat), chat = resultChat, bot=resultBot, time=resultTime)
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://b871-34-86-219-88.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [12/Jun/2022 03:31:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2022 03:31:42] "GET /static/https://ajax.googleapis.com/ajax/libs/jquery/3.3.1/jquery.min.js HTTP/1.1" 404 -
127.0.0.1 - - [12/Jun/2022 03:31:42] "GET /assets/css/style.css HTTP/1.1" 404 -
127.0.0.1 - - [12/Jun/2022 03:31:42] "GET /static/https://cdnjs.cloudflare.com/ajax/libs/malihu-custom-scrollbar-plugin/3.1.5/jquery.mCustomScrollbar.min.js HTTP/1.1" 404 -
127.0.0.1 - - [12/Jun/2022 03:31:45] "GET /favicon.ico HTTP/1.1" 404 -


halo


127.0.0.1 - - [12/Jun/2022 03:31:48] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2022 03:31:49] "GET /assets/css/style.css HTTP/1.1" 404 -
127.0.0.1 - - [12/Jun/2022 03:31:49] "GET /static/https://cdnjs.cloudflare.com/ajax/libs/malihu-custom-scrollbar-plugin/3.1.5/jquery.mCustomScrollbar.min.js HTTP/1.1" 404 -
127.0.0.1 - - [12/Jun/2022 03:31:49] "GET /static/https://ajax.googleapis.com/ajax/libs/jquery/3.3.1/jquery.min.js HTTP/1.1" 404 -
127.0.0.1 - - [12/Jun/2022 03:31:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2022 03:31:55] "GET /static/https://ajax.googleapis.com/ajax/libs/jquery/3.3.1/jquery.min.js HTTP/1.1" 404 -
127.0.0.1 - - [12/Jun/2022 03:31:55] "GET /assets/css/style.css HTTP/1.1" 404 -
127.0.0.1 - - [12/Jun/2022 03:31:55] "GET /static/https://cdnjs.cloudflare.com/ajax/libs/malihu-custom-scrollbar-plugin/3.1.5/jquery.mCustomScrollbar.min.js HTTP/1.1" 404 -
127.0.0.1 - - [12/Jun/2022 03:32:01] "POST /predict HTTP/1.1" 200 -


ba



127.0.0.1 - - [12/Jun/2022 03:32:01] "GET /static/https://ajax.googleapis.com/ajax/libs/jquery/3.3.1/jquery.min.js HTTP/1.1" 404 -
127.0.0.1 - - [12/Jun/2022 03:32:01] "GET /assets/css/style.css HTTP/1.1" 404 -
127.0.0.1 - - [12/Jun/2022 03:32:01] "GET /static/https://cdnjs.cloudflare.com/ajax/libs/malihu-custom-scrollbar-plugin/3.1.5/jquery.mCustomScrollbar.min.js HTTP/1.1" 404 -


In [ ]:
pytz.all_timezones

['Africa/Abidjan',
 'Africa/Accra',
 'Africa/Addis_Ababa',
 'Africa/Algiers',
 'Africa/Asmara',
 'Africa/Asmera',
 'Africa/Bamako',
 'Africa/Bangui',
 'Africa/Banjul',
 'Africa/Bissau',
 'Africa/Blantyre',
 'Africa/Brazzaville',
 'Africa/Bujumbura',
 'Africa/Cairo',
 'Africa/Casablanca',
 'Africa/Ceuta',
 'Africa/Conakry',
 'Africa/Dakar',
 'Africa/Dar_es_Salaam',
 'Africa/Djibouti',
 'Africa/Douala',
 'Africa/El_Aaiun',
 'Africa/Freetown',
 'Africa/Gaborone',
 'Africa/Harare',
 'Africa/Johannesburg',
 'Africa/Juba',
 'Africa/Kampala',
 'Africa/Khartoum',
 'Africa/Kigali',
 'Africa/Kinshasa',
 'Africa/Lagos',
 'Africa/Libreville',
 'Africa/Lome',
 'Africa/Luanda',
 'Africa/Lubumbashi',
 'Africa/Lusaka',
 'Africa/Malabo',
 'Africa/Maputo',
 'Africa/Maseru',
 'Africa/Mbabane',
 'Africa/Mogadishu',
 'Africa/Monrovia',
 'Africa/Nairobi',
 'Africa/Ndjamena',
 'Africa/Niamey',
 'Africa/Nouakchott',
 'Africa/Ouagadougou',
 'Africa/Porto-Novo',
 'Africa/Sao_Tome',
 'Africa/Timbuktu',
 'Africa/